In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import pandas as pd
import numpy as np
import sklearn as sk
print('pandas version: ',pd.__version__)
print('numpy version:',np.__version__)
print('sklearn version:',sk.__version__)

pandas version:  0.20.3
numpy version: 1.13.3
sklearn version: 0.19.1


The data we will analyze today is from the weather database.

In [15]:
colspecs = [(0, 11), (11, 21), (21, 31), (31, 38),(38,41),(41,72),(72,76),(76,80),(80,86)]
stations = pd.read_fwf('ghcnd-stations_buffered.txt', colspecs=colspecs, header=None, index_col=0,
                       names=['latitude','longitude','elevation','state','name','GSNFLAG','HCNFLAG','WMOID'])
stations.index.name='station'
stations=stations[['latitude', 'longitude', 'elevation', 'state', 'name']]
print(stations.shape)
stations.head()

(85284, 5)


,latitude,longitude,elevation,state,name
station,,,,,
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP
AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG
AG000060390,36.7167,3.2500,24.0,NaN,ALGER-DAR EL BEIDA


In [33]:
stations.to_csv('all_stations.tsv',sep='\t')

In [35]:
!head all_stations.tsv

station	latitude	longitude	elevation	state	name
ACW00011604	17.1167	-61.7833	10.1		ST JOHNS COOLIDGE FLD
ACW00011647	17.1333	-61.7833	19.2		ST JOHNS
AE000041196	25.333	55.517	34.0		SHARJAH INTER. AIRP
AF000040930	35.317	69.017	3366.0		NORTH-SALANG
AG000060390	36.7167	3.25	24.0		ALGER-DAR EL BEIDA
AG000060590	30.5667	2.8667	397.0		EL-GOLEA
AG000060611	28.05	9.6331	561.0		IN-AMENAS
AG000060680	22.8	5.4331	1362.0		TAMANRASSET
AGE00135039	35.7297	0.65	50.0		ORAN-HOPITAL MILITAIRE


In [34]:
!grep \, all_stations.tsv

AJ000037742	40.9	47.3	313.0		ORDJONIKIDZE,ZERNOSOVHOZ
AJ000037869	40.3	50.6	-17.0		JILOY,OSTROV
AJ000037888	39.5	45.8	3282.0		SISIANSKIY,PEREVAL
AJ000037902	39.8	47.5	157.0		MOL'SKAYA,STEP'
AJ000037922	39.8	49.6	-25.0		SVINOY,OSTROV
AM000037694	40.9	44.4	2061.0		PUSHKINSKIY,PEREVAL
AM000037878	39.8	45.0	2130.0		BESIIMYANNIY,PEREVAL
ASN00023043	-34.7607	138.6311	25.0		HALBURY ROAD, SALISBURY (HALBU
BR00C4-019A	-22.0	-47.67	800.0		FAZ.    MONT,ALVERNE
CA006106398	45.95	-77.317	130.0		PETAWAWA A,ONT
CA008403045	47.62	-52.83	137.0		PORTUGAL COVE, CONCEPTION BAY
GG000037495	41.73	41.73	30.0		CHAKVA,AGRO
GG000037507	41.6	42.5	2025.0		GODERDZSCIY,PEREVAL
GP000078897	16.27	-61.6	11.0		LE RAIZET, GUADELOUPE
KG000038608	40.0	70.9	1050.0		BATKEN,AMSG
KZ000028856	53.6	64.3	178.0		MIHAYLOVKA,KUSTANAY
KZ000028868	53.3	67.0	190.0		CHERBONNIY,SOVHOZ
KZ000028888	53.9	71.5	123.0		KZILTUSSKIY,SOVHOZ
KZ000028894	53.7	73.2	69.0		OZERNIY,SOVHOZ
KZ000028997	52.7	73.7	89.0		KRASNOKUTSKIY,ZERNOSOVHOZ
KZ0000297

In [20]:
dist_to_coast=pd.read_csv('station_lat_log_elev_distToCoast.csv').set_index('station')
dist_to_coast=dist_to_coast[['dist_coast']]
print(dist_to_coast.shape)
dist_to_coast.head()


(12140, 1)


,dist_coast
station,
USC00044534,107.65500
USC00356784,0.61097
USC00243581,1316.54000
USC00205601,685.50100
CA005010480,908.27100


In [21]:
joined=dist_to_coast.join(stations,how='left',lsuffix='_dist',rsuffix='_orig')
print(joined.shape)
joined.head()

(12140, 6)


,dist_coast,latitude,longitude,elevation,state,name
station,,,,,,
USC00044534,107.65500,36.0042,-119.9600,73.2,CA,KETTLEMAN CITY
USC00356784,0.61097,42.7519,-124.5011,12.8,OR,PORT ORFORD NO 2
USC00243581,1316.54000,47.1064,-104.7183,632.8,MT,GLENDIVE
USC00205601,685.50100,41.7500,-84.2167,247.2,MI,MORENCI
CA005010480,908.27100,49.9170,-99.9500,409.0,NaN,BRANDON AIRPORT


In [28]:
joined.to_csv('US_stations.tsv',sep='\t')

In [32]:
!grep \, US_stations.tsv

CA006106398	479.51300000000003	45.95	-77.317	130.0		PETAWAWA A,ONT


In [30]:
read=pd.read_csv('US_stations.tsv',sep='\t',index_col='station')
read.head()

,dist_coast,latitude,longitude,elevation,state,name
station,,,,,,
USC00044534,107.65500,36.0042,-119.9600,73.2,CA,KETTLEMAN CITY
USC00356784,0.61097,42.7519,-124.5011,12.8,OR,PORT ORFORD NO 2
USC00243581,1316.54000,47.1064,-104.7183,632.8,MT,GLENDIVE
USC00205601,685.50100,41.7500,-84.2167,247.2,MI,MORENCI
CA005010480,908.27100,49.9170,-99.9500,409.0,NaN,BRANDON AIRPORT
